<a href="https://colab.research.google.com/github/taznica/ComputerVision_Assignments/blob/main/assignment_2_data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [512]:
#####################################################
### --- We modify the dataloader in this block ---###
#####################################################

from __future__ import print_function
import torchvision.datasets.vision as vision
import warnings
from PIL import Image
import os
import os.path
import random
import numpy as np
import torch
import codecs
# from torchvision.datasets.utils import download_url, download_and_extract_archive, extract_archive, \
#    makedir_exist_ok, verify_str_arg
from torchvision.datasets.utils import download_url, download_and_extract_archive, extract_archive, \
    verify_str_arg

# This class is copied from torchvision.datasets.MNIST
class MyMNIST(vision.VisionDataset):

    def __init__(self, root, train=True, transform=None, target_transform=None,
                 download=False, limit_data=None):
        super(MyMNIST, self).__init__(root, transform=transform,
                                    target_transform=target_transform)
        self.train = train  # training set or test set

        # ------- We can ingore this block -------
        if download:
            self.download()

        if not self._check_exists():
            raise RuntimeError('Dataset not found.' +
                               ' You can use download=True to download it')
        if self.train:
            data_file = self.training_file
        else:
            data_file = self.test_file
        # -----------------------------------------------

        # We change the lines below; these specify how the data are loaded.
        # self.data contain images (type 'torch.Tensor' of [num_images, H, W]) 
        # self.targets contain labels (class ids) (type 'torch.Tensor' of [num_images]) 
        # images and labels are stored in the same 
        self.data, self.targets = torch.load(os.path.join(self.processed_folder, data_file))

        # We use only the images and lables whose indeces are in the range of 0..limit_data-1.
        if not limit_data is None:
          self.data    = self.data[   :limit_data, :,:]
          self.targets = self.targets[:limit_data]
          if self.train:
            print("[WRN]: Trainig Data is limited, only the first "+str(self.data.size(0))+" samples will be used.")
          else:
            print("[WRN]: Test Data is limited, only the first "   +str(self.data.size(0))+" samples will be used.")            


    def __getitem__(self, index):
        # We extract the image and label of the specified 'index'.
        img, target = self.data[index], int(self.targets[index])

        # Prepare for self.transform below.
        img = Image.fromarray(img.numpy(), mode='L')

        # Transform img.
        if self.transform is not None:
            img = self.transform(img)

        return img, target

    def __len__(self):
        return len(self.data)

# ---------------------- We just leave the code from here as original -----------------
    """`MNIST <http://yann.lecun.com/exdb/mnist/>`_ Dataset.

    Args:
        root (string): Root directory of dataset where ``MNIST/processed/training.pt``
            and  ``MNIST/processed/test.pt`` exist.
        train (bool, optional): If True, creates dataset from ``training.pt``,
            otherwise from ``test.pt``.
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
    """

    urls = [
        'http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
        'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
        'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
        'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz',
    ]
    training_file = 'training.pt'
    test_file = 'test.pt'
    classes = ['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four',
               '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']

    @property
    def train_labels(self):
        warnings.warn("train_labels has been renamed targets")
        return self.targets

    @property
    def test_labels(self):
        warnings.warn("test_labels has been renamed targets")
        return self.targets

    @property
    def train_data(self):
        warnings.warn("train_data has been renamed data")
        return self.data

    @property
    def test_data(self):
        warnings.warn("test_data has been renamed data")
        return self.data

    @property
    def raw_folder(self):
        return os.path.join(self.root, self.__class__.__name__, 'raw')

    @property
    def processed_folder(self):
        return os.path.join(self.root, self.__class__.__name__, 'processed')

    @property
    def class_to_idx(self):
        return {_class: i for i, _class in enumerate(self.classes)}

    def _check_exists(self):
        return (os.path.exists(os.path.join(self.processed_folder,
                                            self.training_file)) and
                os.path.exists(os.path.join(self.processed_folder,
                                            self.test_file)))
        
    def download(self):
        """Download the MNIST data if it doesn't exist in processed_folder already."""

        if self._check_exists():
            return

        # makedir_exist_ok(self.raw_folder)
        # makedir_exist_ok(self.processed_folder)
        os.makedirs(self.raw_folder, exist_ok=True)
        os.makedirs(self.processed_folder, exist_ok=True)

        # download files
        for url in self.urls:
            filename = url.rpartition('/')[2]
            download_and_extract_archive(url, download_root=self.raw_folder, filename=filename)

        # process and save as torch files
        print('Processing...')

        training_set = (
            read_image_file(os.path.join(self.raw_folder, 'train-images-idx3-ubyte')),
            read_label_file(os.path.join(self.raw_folder, 'train-labels-idx1-ubyte'))
        )
        test_set = (
            read_image_file(os.path.join(self.raw_folder, 't10k-images-idx3-ubyte')),
            read_label_file(os.path.join(self.raw_folder, 't10k-labels-idx1-ubyte'))
        )
        with open(os.path.join(self.processed_folder, self.training_file), 'wb') as f:
            torch.save(training_set, f)
        with open(os.path.join(self.processed_folder, self.test_file), 'wb') as f:
            torch.save(test_set, f)

        print('Done!')

    def extra_repr(self):
        return "Split: {}".format("Train" if self.train is True else "Test")

def read_label_file(path):
    with open(path, 'rb') as f:
        x = read_sn3_pascalvincent_tensor(f, strict=False)
    assert(x.dtype == torch.uint8)
    assert(x.ndimension() == 1)
    return x.long()


def read_image_file(path):
    with open(path, 'rb') as f:
        x = read_sn3_pascalvincent_tensor(f, strict=False)
    assert(x.dtype == torch.uint8)
    assert(x.ndimension() == 3)
    return x

def get_int(b):
    return int(codecs.encode(b, 'hex'), 16)


def open_maybe_compressed_file(path):
    """Return a file object that possibly decompresses 'path' on the fly.
       Decompression occurs when argument `path` is a string and ends with '.gz' or '.xz'.
    """
    if not isinstance(path, torch._six.string_classes):
        return path
    if path.endswith('.gz'):
        import gzip
        return gzip.open(path, 'rb')
    if path.endswith('.xz'):
        import lzma
        return lzma.open(path, 'rb')
    return open(path, 'rb')


def read_sn3_pascalvincent_tensor(path, strict=True):
    """Read a SN3 file in "Pascal Vincent" format (Lush file 'libidx/idx-io.lsh').
       Argument may be a filename, compressed filename, or file object.
    """
    # typemap
    if not hasattr(read_sn3_pascalvincent_tensor, 'typemap'):
        read_sn3_pascalvincent_tensor.typemap = {
            8: (torch.uint8, np.uint8, np.uint8),
            9: (torch.int8, np.int8, np.int8),
            11: (torch.int16, np.dtype('>i2'), 'i2'),
            12: (torch.int32, np.dtype('>i4'), 'i4'),
            13: (torch.float32, np.dtype('>f4'), 'f4'),
            14: (torch.float64, np.dtype('>f8'), 'f8')}
    # read
    with open_maybe_compressed_file(path) as f:
        data = f.read()
    # parse
    magic = get_int(data[0:4])
    nd = magic % 256
    ty = magic // 256
    assert nd >= 1 and nd <= 3
    assert ty >= 8 and ty <= 14
    m = read_sn3_pascalvincent_tensor.typemap[ty]
    s = [get_int(data[4 * (i + 1): 4 * (i + 2)]) for i in range(nd)]
    parsed = np.frombuffer(data, dtype=m[1], offset=(4 * (nd + 1)))
    assert parsed.shape[0] == np.prod(s) or not strict
    return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)

In [513]:
#####################################################
### --- Define a simple network in this block --- ###
#####################################################

import torch
import torch.nn as nn
import torch.nn.functional as F

# Build a simple network
class simple_network(nn.Module):
  def __init__(self, input_dim=1, num_class=10):
    super(simple_network, self).__init__()

    # Fully connected layers
    self.fc1 = nn.Linear(784, num_class)
    # self.fc1 = nn.Linear(784, 512)
    #nn.init.kaiming_uniform_(self.fc1.weight)
    # self.fc2 = nn.Linear(512, num_class)
    #nn.init.kaiming_uniform_(self.fc2.weight)
    # self.fc3 = nn.Linear(196, num_class)

    # Convolutional layers
    # self.conv1 = nn.Conv2d(input_dim, 20,  kernel_size=5, stride=1, padding=0)
    # self.conv2 = nn.Conv2d(    20,    50,  kernel_size=5, stride=1, padding=0)
    
    # Activation func.
    # self.relu = nn.ReLU()

    # self.dropout1 = nn.Dropout(0.2)
    
  def forward(self, x):
    # x = self.relu(self.fc1(x))                   # print("Trainig Data is limited, only the first "+str(self.data.size(0))+" samples are used.")
    # x = self.relu(self.fc2(x))     
    x = F.relu(self.fc1(x))  
    # x = self.dropout1(x)
    # x = F.relu(self.fc2(x))   
    # x = F.relu(self.conv1(x))
    # x = F.relu(self.fc3(x))

    return x

In [514]:
##########################################################################
### Prepare the trainloader/testloader for training/validating network ###
##########################################################################

from   torchvision import datasets as datasets
import torchvision.transforms as transforms
import torch.utils as utils
import matplotlib.pyplot as plt
import torch
import torchvision

# The 「transform」 is used to 
# i)  convert PIL.Image to torch.FloatTensor (batch, dim, H, W), and change the 
#     inputs' range to [0, 1] (by inputs/= 255.0);
# ii) standardize the input images by mean=0.1307, std=0.3081

# transform = transforms.Compose([transforms.ToTensor(),
#                                 transforms.Normalize((0.1307,), (0.3081,))])
# transform = transforms.Normalize((0.1307,), (0.3081,))
transform = transforms.Compose([
                                # transforms.RandomAffine(degrees=[-10.0, 10.0]),
                                # transforms.RandomPerspective(),
                                # transforms. RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                # transforms.RandomErasing(scale=(0.02, 0.33)),
                                # transforms.GaussianBlur(kernel_size=1),
                                transforms.Normalize((0.1307,), (0.3081,)),
                                ])
print(transform)

# Initialize dataloaders.
limit_data = 1000 # The first $limit_data samples
mnist_train = MyMNIST('./data', train=True,  download=True, transform=transform, limit_data=limit_data)
mnist_test  = MyMNIST('./data', train=False, download=True, transform=transform)
trainloader = utils.data.DataLoader(mnist_train, batch_size=50, shuffle=True,  num_workers=2)
testloader  = utils.data.DataLoader(mnist_test, batch_size=100, shuffle=False, num_workers=2)

Compose(
    RandomHorizontalFlip(p=0.5)
    ToTensor()
)
[WRN]: Trainig Data is limited, only the first 1000 samples will be used.


In [515]:
# Initialize the network
net = simple_network().cuda()
# net = LeNet().cuda()

print(net)

simple_network(
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)


In [516]:
############################################
### Prepare the optimizer and loss func. ###
############################################

import torch.optim as optim
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0)
# optimizer = optim.Adam(net.parameters())
epoch = 0

In [517]:
######################################################
#### ------ Script for evaluate the network ------ ###
######################################################

def evaluate_model():
  print("Testing the network...")
  net.eval()
  total_num   = 0
  correct_num = 0
  for test_iter, test_data in enumerate(testloader):
    # Get one batch of test samples
    inputs, labels = test_data    
    bch = inputs.size(0)
    inputs = inputs.view(bch, -1)

    # Move inputs and labels into GPU
    inputs = inputs.cuda()
    labels = torch.LongTensor(list(labels)).cuda()

    # Forward
    outputs = net(inputs)   

    # Get predicted classes
    _, pred_cls = torch.max(outputs, 1)
    # if total_num == 0:
    #    print("True label:\n", labels)
    #    print("Prediction:\n", pred_cls)
    # # Record test result
    correct_num+= (pred_cls == labels).float().sum().item()
    total_num+= bch
  net.train()
  
  print("Accuracy: "+"%.2f"%(correct_num/float(total_num)))

In [518]:
#####################################################
### ------ Script for training the network ------ ###
#####################################################

epoch_size = 10

# add
total_num   = 0
correct_num = 0
# add-end

for epoch_idx in range(epoch_size):
  running_loss = 0.0
  ct_num       = 0
  for iteration, data in enumerate(trainloader):
    # Take the inputs and the labels for 1 batch.
    inputs, labels = data
    bch = inputs.size(0)
    inputs = inputs.view(bch, -1)
    
    # Move inputs and labels into GPU
    inputs = inputs.cuda()
    labels = labels.cuda()

    # Remove old gradients for the optimizer.
    optimizer.zero_grad()

    # Compute result (Forward)
    outputs = net(inputs)

    # Compute loss
    loss    = loss_func(outputs, labels)

    # Calculate gradients (Backward)
    loss.backward()

    # Update parameters
    optimizer.step()
    
    #with torch.no_grad():
    running_loss += loss.item()
    ct_num+= 1
    if iteration%20 == 19:
      # add
      # _, pred_cls = torch.max(outputs, 1)
      # correct_num+= (pred_cls == labels).float().sum().item()
      # total_num+= bch
      # add-end
      print("[Epoch: "+str(epoch+1)+"]"" --- Iteration: "+str(iteration+1)+", Loss: "+str(running_loss/ct_num)+'.')

    # Test
    if epoch%3 == 2 and iteration%100 == 99:
      evaluate_model()
    
  epoch += 1

[Epoch: 1] --- Iteration: 20, Loss: 2.2825344800949097.
[Epoch: 2] --- Iteration: 20, Loss: 2.1922790884971617.
[Epoch: 3] --- Iteration: 20, Loss: 2.086781072616577.
[Epoch: 4] --- Iteration: 20, Loss: 1.992095011472702.
[Epoch: 5] --- Iteration: 20, Loss: 1.9028127431869506.
[Epoch: 6] --- Iteration: 20, Loss: 1.8185592234134673.
[Epoch: 7] --- Iteration: 20, Loss: 1.7372877597808838.
[Epoch: 8] --- Iteration: 20, Loss: 1.6702316761016847.
[Epoch: 9] --- Iteration: 20, Loss: 1.6013930797576905.
[Epoch: 10] --- Iteration: 20, Loss: 1.5402338624000549.


In [519]:
# print("Train Accuracy: "+"%.2f"%(correct_num/float(total_num)))
evaluate_model()

Testing the network...
Accuracy: 0.64
